In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Project: One-Mass Oscillator Optimization

## Introduction

In this project, various optimization algorithms will be applied to fit a one-mass oscillator model to real-world data. The objective is to minimize the sum of the squared residuals between the model predictions and the observed amplitudes of a one-mass oscillator system across different frequencies.

### One-Mass Oscillator Model

The one-mass oscillator is characterized by the following equation, representing the amplitudes of the system:

$$ V(\omega) = \frac{F}{\sqrt{(1 - \nu^2)^2 + 4D^2\nu^2}} $$

Here, 
- $ \omega $ represents the angular frequency of the system,
- $ \nu $ is the ratio of the excitation frequency to the natural frequency ($ \nu = \frac{\omega_{\text{err}}}{\omega_{\text{eig}}} $),
- $ D $ is the damping ratio,
- $ F $ is the force applied to the system.

The goal of the project is to determine the optimal values for the parameters $ \omega_{\text{eig}} $, $ D $, and $ F $ that result in the best fit of the one-mass oscillator model to the observed amplitudes.

### Load the real world data

- we have two different measurements
- J represents the measured frequencies
- N represents the measured amplitudes

In [13]:
df1 = pd.read_pickle("./data/df1.pkl")
df2 = pd.read_pickle("./data/df2.pkl")

### Low amplitudes distort the fit and are negligible therefore we define a lower threshold for N

In [14]:
# Define maximum threshold for removing low amplitudes
max_threshold = 0.4 * max(df1["N"])

# Remove the low amplitude entries to avoid distortion in fitting
df1 = df1[df1["N"]>=max_threshold]
df2 = df2[df2["N"]>=max_threshold]

### We extract the frequency value for maximum value of the amplitude. This serves as the initial value for one decision variable

In [15]:
df1_max = df1[df1["N"]==max(df1["N"])]
df1_initial_eig = df1_max["J"].values[0]
df1_max_N = df1_max["N"].values[0]

df2_max = df2[df2["N"]==max(df2["N"])]
df2_initial_eig = df2_max["J"].values[0]
df2_max_N = df2_max["N"].values[0]

### We also have to define the other two initial guesses

In [16]:
# Initial guesses of force and damping ratio values
initial_D = 0.006
initial_F = 0.120

df1_initial_values = [df1_initial_eig, initial_D, initial_F]
df2_initial_values = [df2_initial_eig, initial_D, initial_F]

### Additionally we define the bounds for the decision variables

In [8]:
# Define minimum and maximum frequency values for boundary condition
df1_min_J = min(df1["J"])
df1_max_J = max(df1["J"])

df2_min_J = min(df2["J"])
df2_max_J = max(df2["J"])

In [9]:
df1_bounds = [(df1_min_J, df1_max_J), (0, 0.03), (0, 1)]
df2_bounds = [(df2_min_J, df2_max_J), (0, 0.03), (0, 1)]

### Then we define our one mass oscialltor and objective function

In [10]:
def one_mass_oscillator(params, J) -> np.ndarray:
    # returns amplitudes of the system
    # Defines the model of a one mass oscilator 
    eig, D, F = params
    nue = J / eig
    V = F / (np.sqrt((1 - nue**2) ** 2 + (4 * D**2 * nue**2)))
    return V

In [11]:
def objective_function(params, J, N) -> np.ndarray:
    # sum of squared errors to compare calculated and real amplitudes
    return np.sum((N - one_mass_oscillator(params, J)) ** 2)

### Define gradient function of objective function for optimizers that require gradient matrix 

In [21]:
def gradient(params, J, N):
    eig, D, F = params
    nue = J / eig

    u = (1 - nue**2) ** 2 + (4 * D**2 * nue**2)
    dfdF = np.sum(-2 / np.sqrt(u))
    dfdD = np.sum((8 * F * D * nue**2) / (np.sqrt(u ** 3)))
    dfdeig = np.sum((4 * (nue**2 - nue**4 - 2 * nue**2 * D**2)) / (eig * np.sqrt(u ** 3)))

    return np.array([dfdF, dfdD, dfdeig])

### Define hessian function of objective function for optimizers that require hessian matrix 

In [30]:
def hessian(params, J, N):
    eig, D, F = params
    nue = J / eig

    u = (1 - nue**2) ** 2 + (4 * D**2 * nue**2)
    du = (nue**2 - nue**4 - 2 * nue**2 * D**2)
    d2fdF = 0
    d2fdFD = np.sum((8 * D * nue**2) / (np.sqrt(u ** 3)))
    d2fdFeig = np.sum((4 * du) / (eig * np.sqrt(u ** 3)))
    d2fdDF = d2fdFD
    d2fdD = np.sum(8 * nue**2 * F - ((192 * D**2 * nue**4 * F) / u))
    d2fdDeig = np.sum((16 * F * D * nue**2 / eig) * (-1 - (6 * du) / u))
    d2fdeigF = d2fdFeig
    d2fdeigD = d2fdDeig
    d2fdeig = np.sum(((20 * nue**4 - 12 * nue**2 + 24 * nue**2 * D**2) / eig**2)  - ((48 * du ** 2) / (eig * u)))

    return np.array([[d2fdF, d2fdFD, d2fdFeig],[d2fdDF, d2fdD, d2fdDeig],[d2fdeigF, d2fdeigD, d2fdeig]])

In [31]:
# Think about how to implement other optimizers without repeating  the same thing (ex: extract the argumnts as dictionary to another file)
# Get the results of the optimizers
# Generate the data using the results
# Plot the results

array([[ 0.00000000e+00,  8.34061075e+05,  7.12324860e+00],
       [ 8.34061075e+05, -1.44836989e+02, -1.55022263e-03],
       [ 7.12324860e+00, -1.55022263e-03, -3.73859490e-03]])